In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, accuracy_score, confusion_matrix, roc_curve,roc_auc_score,auc, f1_score
from sklearn.feature_selection import SelectFromModel


In [2]:
df_member = pd.read_csv("member.csv", encoding= 'UTF-16',sep = '\t')
df_member.head()

,id,is_vip,gender,birthdate,city,has_google_id,has_yahoo_id,has_weibo_id,cdate
0,e732bcf914fecc5a20adc8940689e701160772c4,0,F,5/14/1981,台北市,0,0,0,4/1/2012 0:01
1,c671d1bd0b03d682d590275f929b613426dc0e34,0,M,3/22/1975,台北市,0,0,0,1/1/2012 1:22
2,fe65711316d561f420bfa3364edf60f0c1859d1f,0,F,11/1/1976,0,0,0,0,1/1/2012 1:32
3,6113c28cfc2cf86ae268657683eca4953df54342,0,M,0000-00-00,0,0,0,0,1/1/2012 1:33
4,2fc08e00a891d491c99323aed57c4cb53922c959,0,F,1/1/1977,0,0,0,0,1/1/2012 1:54


In [3]:
df_restau = pd.read_csv("restaurant.csv", encoding = 'UTF-16', sep='\t')
df_restau.head()

,id,is_hotel,country,currency,city,cityarea,name,abbr,tel,opening_hours,...,outdoor_seating,wifi,wheelchair_accessible,price1,price2,lat,lng,timezone,locale,cdate
0,fb1171550a7ad65f5533d24345e7677c5353168d,1,tw,TWD,新北市,NaN,db231778a1d2a260b510e4379dfb159a63e90e95,e0199e43c23c7322f91049f1c49a9c8d87b3bc39,50f5512c6ed9d5e1b8112e7e52fa3d2aa76514ea,24小時營業(湯屋每次為雙人使用，加時與加人依現場報價為準),...,0,1,1,1100,2200,25.20,121.60,Asia/Taipei,zh_TW,2/27/2013 18:25
1,8239c2381e09ba5a298eb9ebefa88de9976105a5,0,tw,TWD,新北市,NaN,da4531bc68b42dff77fffc7f1b748bfd2a1855b7,da4531bc68b42dff77fffc7f1b748bfd2a1855b7,44d48f31e6b7aa2184018b7d0156fd8e068e3368,週二至週日\r\n午餐：11:00~14:00 \r\n晚餐：17:00~ 21:00\r\...,...,0,1,0,300,400,25.00,121.52,Asia/Taipei,zh_TW,8/26/2014 13:37
2,85709eabedeb5c761b424baa16db3bccd6144fd4,1,tw,TWD,台中市,請選擇,e1bb633022c44a15d15a072e17274887707e5774,e1bb633022c44a15d15a072e17274887707e5774,a43a6b1ab17d261040e844ad4db619924c58d188,營業時間：06:30~23:30\r\n\r\n,...,0,1,1,563,736,24.23,120.94,Asia/Taipei,zh_TW,2/25/2012 20:47
3,8a035d8ccfaae36efff1cdcc6f02f318f35abd64,1,tw,TWD,台中市,烏日區,2fb3f01285d5d64069513e2f2a06e322e352dcdc,2fb3f01285d5d64069513e2f2a06e322e352dcdc,353015f4db1660c7060c3e9a3063097aff22efa2,平日早餐：6:30- 10:00; \r\n午餐：11:30- 14:00（14:00清場)...,...,0,1,1,379,564,24.14,120.59,Asia/Taipei,zh_TW,6/25/2012 11:59
4,56a3a7c9675c5aca5e98ea3d9aa77610ec04183a,0,tw,TWD,台中市,NaN,408bce81d566a17affd9b56344e0dfd111c69548,408bce81d566a17affd9b56344e0dfd111c69548,caeaa8587bfb7b04cc0d6c1fcf15099decdba1cd,午餐:11:30~14:00\r\n午茶:14:30~16:30\r\n晚餐:17:30~2...,...,0,1,0,499,1000,24.23,120.94,Asia/Taipei,zh_TW,8/15/2013 16:56


In [4]:
df_train = pd.read_csv('Train.csv', sep=',')
df_train.head()

,booking_id,member_id,cdate,restaurant_id,datetime,people,purpose,gender,status,is_required_prepay_satisfied,return90
0,154664,ec9352fdbcb8df88fbf98f964431c063d226e96e,"1/17/2014,,2:54:00,AM",dea8ce93bfe3b40da0cca953fd9d30811d680373,"1/23/2014,,3:00:00,PM",2,Sweet day,F,ok,1,1
1,99188,97fa82b7632705e970530cacebfe841e908d04b6,"4/28/2012,,11:10:00,PM",f3837c4489cc6c4587dfb8a92a060585c182ccc1,"4/30/2012,,11:30:00,AM",2,Dîner entre amis,M,ok,1,0
2,142140,d9643f4373b465fd8ca5fe75fffdbfd161f7a8df,"2/1/2013,,5:25:00,PM",c460d6b5860c01ad97d3ef5d9886548361e5c78e,"2/4/2013,,7:00:00,PM",2,Dîner entre amis,M,ok,1,0
3,128744,c4baf4ebccdf1a7983ca34673f4b06d995a6eae9,"8/25/2012,,1:53:00,PM",52f8e137046607186800ca3c01a954e928c16412,"8/25/2012,,6:30:00,PM",9,Repas en famille,F,ok,1,0
4,134126,cd16b9904eeee3585eb83d1d19e8ffbeee2b0327,"11/24/2013,,10:47:00,AM",fe230aa38ebf31c978d0f58bd6fd5ab9e7ec1e66,"12/8/2013,,3:00:00,PM",4,Repas en famille,F,canceled,1,1


In [5]:
df_test = pd.read_csv('Test.csv', sep=',')
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38992 entries, 0 to 38991
Data columns (total 10 columns):
booking_id                      38992 non-null int64
member_id                       38992 non-null object
cdate                           38992 non-null object
restaurant_id                   38992 non-null object
datetime                        38992 non-null object
people                          38992 non-null int64
purpose                         36286 non-null object
gender                          38992 non-null object
status                          38992 non-null object
is_required_prepay_satisfied    38992 non-null int64
dtypes: int64(3), object(7)
memory usage: 3.0+ MB


In [6]:
df_test['isTest'] = 1
df_train['isTest'] = 0

In [7]:
#transformation de la dataframe pour fit au model

from sklearn.impute import SimpleImputer


#Pour pouvoir éviter l'erreur, faire un get dummies sur le join du test et du train comme ca il connait toutes les valeurs
# voila voila


print(df_test.info())


imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

df_test_transformed = pd.DataFrame(imp.fit_transform(df_test))


df_test_transformed.columns = ['booking_id','member_id','cdate','restaurant_id','datetime','people','purpose','gender','status','is_required_prepay_satisfied','isTest']



print(df_test_transformed.info())


#for i in range(len(df_test_transformed.index)) :
#    if(df_test_transformed.iloc[i].isnull().sum()>0):
#        print("Nan in row ", i , " : " ,  df_test_transformed.iloc[i].isnull().sum())



#/!\ remplacement des valeurs manquantes par la méthode du most frequent de scikit
#columns renommer pour correspondre à celle du train

#à la sortie de cette cellule
# df_test_transformed sans NaN avec les bons noms de columns 



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38992 entries, 0 to 38991
Data columns (total 11 columns):
booking_id                      38992 non-null int64
member_id                       38992 non-null object
cdate                           38992 non-null object
restaurant_id                   38992 non-null object
datetime                        38992 non-null object
people                          38992 non-null int64
purpose                         36286 non-null object
gender                          38992 non-null object
status                          38992 non-null object
is_required_prepay_satisfied    38992 non-null int64
isTest                          38992 non-null int64
dtypes: int64(4), object(7)
memory usage: 3.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38992 entries, 0 to 38991
Data columns (total 11 columns):
booking_id                      38992 non-null object
member_id                       38992 non-null object
cdate                        

In [8]:
#transformation de df_train pour eviter de propager de rows avec de NaN
print(df_train.info())


imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

df_train_transformed = pd.DataFrame(imp.fit_transform(df_train))


df_train_transformed.columns = df_train.columns.values



print(df_train_transformed.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77982 entries, 0 to 77981
Data columns (total 12 columns):
booking_id                      77982 non-null int64
member_id                       77982 non-null object
cdate                           77982 non-null object
restaurant_id                   77982 non-null object
datetime                        77982 non-null object
people                          77982 non-null int64
purpose                         72453 non-null object
gender                          77982 non-null object
status                          77982 non-null object
is_required_prepay_satisfied    77982 non-null int64
return90                        77982 non-null int64
isTest                          77982 non-null int64
dtypes: int64(5), object(7)
memory usage: 7.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77982 entries, 0 to 77981
Data columns (total 12 columns):
booking_id                      77982 non-null object
member_id                     

In [9]:
#transformation de df_restau pour eviter de propager de rows avec de NaN

imp_restau = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

df_restau_transformed = pd.DataFrame(imp_restau.fit_transform(df_restau))


df_restau_transformed.columns = df_restau.columns.values
    

print(df_restau_transformed.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 724 entries, 0 to 723
Data columns (total 23 columns):
id                       724 non-null object
is_hotel                 724 non-null object
country                  724 non-null object
currency                 724 non-null object
city                     724 non-null object
cityarea                 724 non-null object
name                     724 non-null object
abbr                     724 non-null object
tel                      724 non-null object
opening_hours            724 non-null object
good_for_family          724 non-null object
accept_credit_card       724 non-null object
parking                  724 non-null object
outdoor_seating          724 non-null object
wifi                     724 non-null object
wheelchair_accessible    724 non-null object
price1                   724 non-null object
price2                   724 non-null object
lat                      724 non-null object
lng                      724 non-null ob

In [10]:

df_join = df_train_transformed.append(df_test_transformed, sort=False)


df_restau_transformed = df_restau_transformed.rename(columns={'id':'restaurant_id'})


df_sub = pd.merge(df_restau_transformed, df_join, on='restaurant_id', how = 'outer')


#count = 0
#for i in range(len(df_sub.index)) :
#    if(df_sub.iloc[i].isnull().sum()>0):
#        count +=1
        #print("Nan in row ", i , " : " ,  df_train.iloc[i].isnull().sum())
        



df_sub = df_sub.drop(['currency','cdate_x','cdate_y','name','abbr','tel','timezone','locale','datetime','member_id'],axis=1)
df_sub = df_sub.drop(['price1','price2','lat','lng'],axis =1)
df_sub = df_sub.drop('restaurant_id',axis=1)


df_sub['return90'].fillna(0, inplace=True)

df_sub_without_test = df_sub.dropna(subset=['return90'])



#process sur le df_sub
imp_sub = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

df_sub_transformed = pd.DataFrame(imp_sub.fit_transform(df_sub))


df_sub_transformed.columns = df_sub.columns.values
    


print(df_sub_transformed.info())
#count = 0
#for i in range(len(df_sub.index)) :
#    if(df_sub.iloc[i].isnull().sum()>0):
#        count +=1
        #print("Nan in row ", i , " : " ,  df_train.iloc[i].isnull().sum())










<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117066 entries, 0 to 117065
Data columns (total 19 columns):
is_hotel                        117066 non-null object
country                         117066 non-null object
city                            117066 non-null object
cityarea                        117066 non-null object
opening_hours                   117066 non-null object
good_for_family                 117066 non-null object
accept_credit_card              117066 non-null object
parking                         117066 non-null object
outdoor_seating                 117066 non-null object
wifi                            117066 non-null object
wheelchair_accessible           117066 non-null object
booking_id                      117066 non-null object
people                          117066 non-null object
purpose                         117066 non-null object
gender                          117066 non-null object
status                          117066 non-null object
is_requir

In [11]:
#df_sub_transformed en csv pour plus avoir à tout calculer à chaque fois

df_sub_transformed.to_csv('df_sub_transformed.csv',sep=';')

In [12]:



df_sub_transformed['return90'] = df_sub_transformed['return90'].astype(np.int64)
df_sub_transformed['booking_id'] = df_sub_transformed['booking_id'].astype(np.int64)
df_sub_transformed['is_hotel'] = df_sub_transformed['is_hotel'].astype(np.int64)
df_sub_transformed['good_for_family'] = df_sub_transformed['good_for_family'].astype(np.int64)
df_sub_transformed['accept_credit_card'] = df_sub_transformed['accept_credit_card'].astype(np.int64)
df_sub_transformed['parking'] = df_sub_transformed['parking'].astype(np.int64)
df_sub_transformed['outdoor_seating'] = df_sub_transformed['outdoor_seating'].astype(np.int64)
df_sub_transformed['wifi'] = df_sub_transformed['wifi'].astype(np.int64)
df_sub_transformed['wheelchair_accessible'] = df_sub_transformed['wheelchair_accessible'].astype(np.int64)
df_sub_transformed['people'] = df_sub_transformed['people'].astype(np.int64)
df_sub_transformed['is_required_prepay_satisfied'] = df_sub_transformed['is_required_prepay_satisfied'].astype(np.int64)
df_sub_transformed['isTest'] = df_sub_transformed['isTest'].astype(np.int64)



df_sub_transformed = pd.get_dummies(df_sub_transformed)

print(df_sub_transformed.head())
print(df_sub_transformed.info())








   is_hotel  good_for_family  accept_credit_card  parking  outdoor_seating  \
0         1                1                   1        1                0   
1         1                1                   1        1                0   
2         1                1                   1        1                0   
3         1                1                   1        1                0   
4         1                1                   1        1                0   

   wifi  wheelchair_accessible  booking_id  people  \
0     1                      1      111664       2   
1     1                      1      120055       2   
2     1                      1       80299       2   
3     1                      1      149766       2   
4     1                      1       45451       2   

   is_required_prepay_satisfied  ...  purpose_其他  purpose_家人&#65533;  \
0                             1  ...           0                   0   
1                             1  ...           0              

In [13]:
# df_sub_transformed avec les dummies en csv pour pas avoir à faire le calcul à chaque fois
df_sub_transformed.to_csv('df_sub_transformed_dummies.csv',sep=';')

In [16]:
#permet de choper seulement les rows de test qui ne contiennent pas de return90
X_sub = df_sub_transformed.loc[df_sub_transformed['isTest']== 1]
#print(X_sub.info())


#process sur le X_sub de test sans les return90
imp_X = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

df_X_sub_transformed = pd.DataFrame(imp_sub.fit_transform(X_sub))


df_X_sub_transformed.columns = X_sub.columns.values

In [17]:

print(df_X_sub_transformed.info())
print(df_X_sub_transformed.shape)

df_X_sub_split = df_X_sub_transformed[:38993]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38992 entries, 0 to 38991
Columns: 688 entries, is_hotel to status_ok
dtypes: int64(688)
memory usage: 204.7 MB
None
(38992, 688)


In [22]:
df_X_sub_split.to_csv('Test_submission_transformed.csv',sep=';')

In [19]:
#mise en place des données de train et séparation de ces données en train et test 


print("shape des données du Train.csv : {}".format(df_sub_transformed.shape))
print("shape des données de Test.csv : {}".format(df_X_sub_split.shape))

pre_select = df_sub_transformed.loc[df_sub_transformed['isTest']== 0]

X = pre_select.drop('return90',axis=1)
y = pre_select['return90']


#à la sorti de cette cellule : les données séparé entre les deux fichiers de test.csv et train.csv

shape des données du Train.csv : (117066, 688)
shape des données de Test.csv : (38992, 688)


In [21]:
pre_select.to_csv('Train_transformed.csv',sep=';')